> Copyright 2022 University of Luxembourg
> 
> Licensed under the Apache License, Version 2.0 (the "License");  
> you may not use this file except in compliance with the License.  
> You may obtain a copy of the License at  
>
>    https://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software  
> distributed under the License is distributed on an "AS IS" BASIS,  
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
> See the License for the specific language governing permissions and  
> limitations under the License.  
>
***

Author: André Stemper (andre.stemper@uni.lu)

***

# Postprocessing - Aggregate informations from multiple experiments into a grid form


Measurement results directory


In [ ]:
measurements_root = './measurements'


In [ ]:
import math
import json
import sys
import pickle
import time
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline


In [ ]:
experiment_datasets =['2022.04.06', '2022.05.18', '2022.05.20', '2022.05.30', '2022.06.01', '2022.06.03', '2022.06.08', '2022.06.15', '2022.06.22', '2022.07.20']
experiment_clock_frequencies = [39,78,146,298]

In [ ]:
aggregated_data = {}


Append data from one experiment located in measurements_directory


In [ ]:
def append_experiment(measurement, measurements_directory, verbose=False):
    global aggregated_data
    # load settings
    try:
        filename = measurements_directory + os.path.sep + 'settings.json'
        with open(filename, 'r') as file:
            settings = json.load(file)
            print("{}>Settings loaded from '{}'".format(measurement, filename))
        if verbose:
            print('{}>Settings:'.format(measurement))
            for k, v in settings.items():
                print(" - {} = {}".format(k, v))
    except Exception as e:
        print("{}>!!! Missing settings: {}".format(measurement, e))
        print("{}>!!! Aborting".format(measurement))
        return
    # load power statistics
    try:
        filename = measurements_directory + os.path.sep + 'power_statistics.json'
        with open(filename, 'r') as file:
            power_statistics = json.load(file)
            print("{}>Power statistics loaded from '{}'".format(
                measurement, filename))
        if verbose:
            print('{}>Power statistics:'.format(measurement))
            for k, v in power_statistics.items():
                print(" - {} = {}".format(k, v))
    except Exception as e:
        print("{}>!!! Missing power statistics: {}".format(measurement, e))
        power_statistics['experiment']['mean'] = -1
        power_statistics['experiment']['std'] = -1
        power_statistics['background']['mean'] = -1
        power_statistics['background']['std'] = -1
        power_statistics['difference']['mean'] = -1
        power_statistics['difference']['std'] = -1

    # load timing statistics
    try:
        filename = measurements_directory + os.path.sep + 'execution_time_statistics.json'
        with open(filename, 'r') as file:
            execution_time_statistics = json.load(file)
            print("{}>Execution time statistics loaded from '{}'".format(
                measurement, filename))
        if verbose:
            print('{}>Execution time statistics:'.format(measurement))
            for k, v in execution_time_statistics.items():
                print(" - {} = {}".format(k, v))
    except Exception as e:
        print("{}>!!! Missing execution time statistics: {}".format(measurement, e))

    # aggregate into dict
    row = {'experiment': settings['timestamp'],
           'dataset': settings['dataset'],
           'range': settings['range'],
           'frequency': float(settings['frequency']),
           'connection_type': settings['connection_type'],
           'limit': settings['limit'],
           'threshold_hold_off': int(settings['threshold_hold_off']),
           'power_experiment_mean': power_statistics['experiment']['mean'],
           'power_experiment_std': power_statistics['experiment']['std'],
           'power_background_mean': power_statistics['background']['mean'],
           'power_background_std': power_statistics['background']['std'],
           'power_difference_mean': power_statistics['difference']['mean'],
           'power_difference_std': power_statistics['difference']['std'],
           'execution_time_mean': execution_time_statistics['execution_time']['mean'],
           'execution_time_median': execution_time_statistics['execution_time']['median'],
           'execution_time_std': execution_time_statistics['execution_time']['std']
           }
    aggregated_data[settings['timestamp']] = row


Aggregate from all experiments


In [ ]:
import os
for file in os.listdir(measurements_root):
    d = os.path.join(measurements_root, file)
    if os.path.isdir(d):
        if file in ['timings', 'histograms']:
            continue
        experiment_timestamp_string = file
        append_experiment(experiment_timestamp_string, d)
        print("---")


**convert into a dataframe**


In [ ]:
aggregated_dataframe = pd.DataFrame.from_dict(aggregated_data, orient='index')
filename = measurements_root + os.path.sep + "aggregated.csv"
aggregated_dataframe.to_csv(filename, sep=';')

## Completed experiments

In [ ]:

# Plot
plt.figure(figsize=(12, 10), dpi=80)

experiments_matrix_value = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=int)
experiments_matrix_text = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=object)

for j, dataset in enumerate(experiment_datasets):
    for i, clock_frequency in enumerate(experiment_clock_frequencies):
        filtered = aggregated_dataframe[(aggregated_dataframe['dataset'] == dataset) & (
            aggregated_dataframe['frequency'] == float(clock_frequency))]
        if len(filtered) == 1:
            experiments_matrix_text[j, i] = filtered['experiment'][0]
            experiments_matrix_value[j, i] = 1
        else:
            experiments_matrix_text[j, i] = '--'
            experiments_matrix_value[j, i] = -1

sns.heatmap(experiments_matrix_value,
            annot=experiments_matrix_text,
            xticklabels=experiment_clock_frequencies,
            yticklabels=experiment_datasets,
            fmt="",
            cmap='RdYlGn',
            center=0,
            cbar=False)

# Decorations
plt.title('Experiments completed', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)


filename = measurements_root + os.path.sep + "experiments_completed.png"
plt.savefig(filename)
filename = measurements_root + os.path.sep + "experiments_completed.pdf"
plt.savefig(filename)

plt.show()


## Difference power

In [ ]:

# Plot
plt.figure(figsize=(12, 10), dpi=80)

experiments_matrix_value = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=float)
experiments_matrix_text = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=object)

for j, dataset in enumerate(experiment_datasets):
    for i, clock_frequency in enumerate(experiment_clock_frequencies):
        filtered = aggregated_dataframe[(aggregated_dataframe['dataset'] == dataset) & (
            aggregated_dataframe['frequency'] == float(clock_frequency))]
        if len(filtered) == 1:
            experiments_matrix_text[j, i] = "{:.5f}".format(
                filtered['power_difference_mean'][0])
            experiments_matrix_value[j, i] = float(
                filtered['power_difference_mean'][0])
        else:
            experiments_matrix_text[j, i] = '--'
            experiments_matrix_value[j, i] = 0

sns.heatmap(experiments_matrix_value,
            annot=experiments_matrix_text,
            xticklabels=experiment_clock_frequencies,
            yticklabels=experiment_datasets,
            fmt="",
            cmap=sns.dark_palette("#0E0", as_cmap=True, reverse=True),

            center=0,
            cbar=True)

# Decorations
plt.title('Experiments power difference [W]', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

filename = measurements_root + os.path.sep + \
    "experiments_difference_power_grid.png"
plt.savefig(filename)
filename = measurements_root + os.path.sep + \
    "experiments_difference_power_grid.pdf"
plt.savefig(filename)

plt.show()



## Experiments total power grid

In [ ]:
# Plot
plt.figure(figsize=(12, 10), dpi=80)

experiments_matrix_value = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=float)
experiments_matrix_text = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=object)

for j, dataset in enumerate(experiment_datasets):
    for i, clock_frequency in enumerate(experiment_clock_frequencies):
        filtered = aggregated_dataframe[(aggregated_dataframe['dataset'] == dataset) & (
            aggregated_dataframe['frequency'] == float(clock_frequency))]
        if len(filtered) == 1:
            experiments_matrix_text[j, i] = "{:.5f}".format(filtered['power_experiment_mean'][0])
            experiments_matrix_value[j, i] = float(filtered['power_experiment_mean'][0])
        else:
            experiments_matrix_text[j, i] = '--'
            experiments_matrix_value[j, i] = 0

sns.heatmap(experiments_matrix_value,
            annot=experiments_matrix_text,
            xticklabels=experiment_clock_frequencies,
            yticklabels=experiment_datasets,
            fmt="",
            cmap=sns.dark_palette("#0E0", as_cmap=True, reverse=True),

            center=0,
            cbar=True)

# Decorations
plt.title('Experiment total mean power [W]', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)


filename = measurements_root + os.path.sep + "experiments_total_mean_power_grid.png"
plt.savefig(filename)
filename = measurements_root + os.path.sep + "experiments_total_mean_power_grid.pdf"
plt.savefig(filename)

plt.show()

# Execution time mean

In [ ]:
# Plot
plt.figure(figsize=(12, 10), dpi=80)

experiments_matrix_value = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=float)
experiments_matrix_text = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=object)

for j, dataset in enumerate(experiment_datasets):
    for i, clock_frequency in enumerate(experiment_clock_frequencies):
        filtered = aggregated_dataframe[(aggregated_dataframe['dataset'] == dataset) & (
            aggregated_dataframe['frequency'] == float(clock_frequency))]
        if len(filtered) == 1:
            experiments_matrix_text[j, i] = "{:.5f}".format(filtered['execution_time_mean'][0])
            experiments_matrix_value[j, i] = float(filtered['execution_time_mean'][0])
        else:
            experiments_matrix_text[j, i] = '--'
            experiments_matrix_value[j, i] = 0

sns.heatmap(experiments_matrix_value,
            annot=experiments_matrix_text,
            xticklabels=experiment_clock_frequencies,
            yticklabels=experiment_datasets,
            fmt="",
            cmap=sns.dark_palette("#0F0", as_cmap=True, reverse=True),
            center=0,
            cbar=True)

# Decorations
plt.title('Execution time mean [s]', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

filename = measurements_root + os.path.sep + "experiments_execution_time_mean_grid.png"
plt.savefig(filename)
filename = measurements_root + os.path.sep + "experiments_execution_time_mean_grid.pdf"
plt.savefig(filename)

plt.show()

**3D**

In [ ]:
experiments_matrix_value = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=float)
experiments_matrix_std = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=float)
experiments_matrix_text = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=object)

for j, dataset in enumerate(experiment_datasets):
    for i, clock_frequency in enumerate(experiment_clock_frequencies):
        filtered = aggregated_dataframe[(aggregated_dataframe['dataset'] == dataset) & (
            aggregated_dataframe['frequency'] == float(clock_frequency))]
        if len(filtered) == 1:
            experiments_matrix_text[j, i] = "{:.5f}".format(filtered['execution_time_mean'][0])
            experiments_matrix_value[j, i] = float(filtered['execution_time_mean'][0]) 
            experiments_matrix_std[j, i] = float(filtered['execution_time_std'][0])

        else:
            experiments_matrix_text[j, i] = '--'
            experiments_matrix_value[j, i] = 0
            experiments_matrix_std[j, i] = 0
w = 0.7

dx=w * len(experiment_clock_frequencies) / len(experiment_datasets)
dy=w # */
dz=experiments_matrix_value.ravel()
dz_std=experiments_matrix_std.ravel()

xpos = np.arange(len(experiment_clock_frequencies))
ypos = np.arange(len(experiment_datasets))
xposM, yposM = np.meshgrid(xpos, ypos, copy=False)

font = {'family': 'arial',
        'color':  'black',
        'weight': 'normal',
        'size': 14,
        }
fig=plt.figure(figsize=(10, 10), dpi=300)
ax1=fig.add_subplot(111, projection='3d')
ax1.set_xlabel('Clock frequency [MHz]', labelpad=10, fontdict=font)
ax1.set_ylabel('Dataset', labelpad=25, fontdict=font)
ax1.set_zlabel('Execution time [s]', labelpad=16, fontdict=font)

ax1.w_xaxis.set_ticks(xpos + dy/2.)
ax1.w_xaxis.set_ticklabels(experiment_clock_frequencies)

ax1.w_yaxis.set_ticks(ypos + 2)
ax1.w_yaxis.set_ticklabels(experiment_datasets)
ax1.set_ylim(0,11)

colors = plt.cm.rainbow(experiments_matrix_value)
ax1.bar3d(xposM.ravel(), yposM.ravel(), dz*0, dx, dy, dz, alpha=1.0 , color='b',edgecolor='b')
ax1.bar3d(xposM.ravel(), yposM.ravel(), dz*1, dx, dy, dz_std, color=(0, 0, 0, 0.1), edgecolor=(0, 0, 0, 0.2)) 

# Decorations
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax1.tick_params('z', labelsize=12)

for tick in ax1.yaxis.get_major_ticks():
    tick.set_pad(10)
for tick in ax1.zaxis.get_major_ticks(): 
    tick.set_pad(10)

plt.title('Execution time mean [s]', fontsize=16)

filename = measurements_root + os.path.sep + "experiments_execution_time_mean_3d_std.png"
plt.savefig(filename)
filename = measurements_root + os.path.sep + "experiments_execution_time_mean_3d_std.pdf"
plt.savefig(filename)

plt.show()

In [ ]:
# Plot
experiments_matrix_value = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=float)
experiments_matrix_std = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=float)
experiments_matrix_text = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=object)

for j, dataset in enumerate(experiment_datasets):
    for i, clock_frequency in enumerate(experiment_clock_frequencies):
        filtered = aggregated_dataframe[(aggregated_dataframe['dataset'] == dataset) & (
            aggregated_dataframe['frequency'] == float(clock_frequency))]
        if len(filtered) == 1:
            experiments_matrix_text[j, i] = "{:.5f}".format(filtered['execution_time_mean'][0])
            experiments_matrix_value[j, i] = float(filtered['execution_time_mean'][0]) 
            experiments_matrix_std[j, i] = float(filtered['execution_time_std'][0])

        else:
            experiments_matrix_text[j, i] = '--'
            experiments_matrix_value[j, i] = 0
            experiments_matrix_std[j, i] = 0


w = 0.7

dx=w * len(experiment_clock_frequencies) / len(experiment_datasets)
dy=w # */
dz=experiments_matrix_value.ravel()
dz_std=experiments_matrix_std.ravel() #+ dz

xpos = np.arange(len(experiment_clock_frequencies))
ypos = np.arange(len(experiment_datasets))
xposM, yposM = np.meshgrid(xpos, ypos, copy=False)

# colors = ['r','b','g','y','b','p']
font = {'family': 'arial',
        'color':  'black',
        'weight': 'normal',
        'size': 14,
        }
fig=plt.figure(figsize=(10, 10), dpi=300)
ax1=fig.add_subplot(111, projection='3d')
ax1.set_xlabel('Clock frequency [MHz]', labelpad=10, fontdict=font)
ax1.set_ylabel('Dataset', labelpad=25, fontdict=font)
ax1.set_zlabel('Execution time [s]', labelpad=16, fontdict=font)

ax1.w_xaxis.set_ticks(xpos + dy/2.)
ax1.w_xaxis.set_ticklabels(experiment_clock_frequencies)

ax1.w_yaxis.set_ticks(ypos + 2)
ax1.w_yaxis.set_ticklabels(experiment_datasets)
ax1.set_ylim(0,11)

colors = plt.cm.rainbow(experiments_matrix_value)
ax1.bar3d(xposM.ravel(), yposM.ravel(), dz*0, dx, dy, dz, alpha=1.0 , color='b',edgecolor='b')

# Decorations
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax1.tick_params('z', labelsize=12)

for tick in ax1.yaxis.get_major_ticks():
    tick.set_pad(10)
for tick in ax1.zaxis.get_major_ticks(): 
    tick.set_pad(10)

plt.title('Execution time mean [s]', fontsize=16)

filename = measurements_root + os.path.sep + "experiments_execution_time_mean_3d.png"
plt.savefig(filename)
filename = measurements_root + os.path.sep + "experiments_execution_time_mean_3d.pdf"
plt.savefig(filename)

plt.show()

## Power difference 3D [W]

In [ ]:
# Plot

experiments_matrix_value = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=float)
experiments_matrix_std = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=float)
experiments_matrix_text = np.zeros(
    (len(experiment_datasets), len(experiment_clock_frequencies)), dtype=object)

for j, dataset in enumerate(experiment_datasets):
    for i, clock_frequency in enumerate(experiment_clock_frequencies):
        filtered = aggregated_dataframe[(aggregated_dataframe['dataset'] == dataset) & (
            aggregated_dataframe['frequency'] == float(clock_frequency))]
        if len(filtered) == 1:
            experiments_matrix_text[j, i] = "{:.5f}".format(filtered['power_difference_mean'][0])
            experiments_matrix_value[j, i] = float(filtered['power_difference_mean'][0]) 
            experiments_matrix_std[j, i] = float(filtered['power_difference_std'][0])

        else:
            experiments_matrix_text[j, i] = '--'
            experiments_matrix_value[j, i] = 0
            experiments_matrix_std[j, i] = 0


w = 0.7

dx=w * len(experiment_clock_frequencies) / len(experiment_datasets)
dy=w # */
dz=experiments_matrix_value.ravel()
dz_std=experiments_matrix_std.ravel()

xpos = np.arange(len(experiment_clock_frequencies))
ypos = np.arange(len(experiment_datasets))
xposM, yposM = np.meshgrid(xpos, ypos, copy=False)

font = {'family': 'arial',
        'color':  'black',
        'weight': 'normal',
        'size': 14,
        }
fig=plt.figure(figsize=(10, 10), dpi=300)
ax1=fig.add_subplot(111, projection='3d')
ax1.set_xlabel('Clock frequency [MHz]', labelpad=10, fontdict=font)
ax1.set_ylabel('Dataset', labelpad=25, fontdict=font)
ax1.set_zlabel('Power difference [W]', labelpad=16, fontdict=font)

ax1.w_xaxis.set_ticks(xpos + dy/2.)
ax1.w_xaxis.set_ticklabels(experiment_clock_frequencies)

ax1.w_yaxis.set_ticks(ypos + 2)
ax1.w_yaxis.set_ticklabels(experiment_datasets)
ax1.set_ylim(0,11)

colors = plt.cm.rainbow(experiments_matrix_value)
ax1.bar3d(xposM.ravel(), yposM.ravel(), dz*0, dx, dy, dz, alpha=1.0 , color='b',edgecolor='b')
ax1.bar3d(xposM.ravel(), yposM.ravel(), dz*1, dx, dy, dz_std, color=(0, 0, 0, 0.1), edgecolor=(0, 0, 0, 0.2)) 

# Decorations
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
ax1.tick_params('z', labelsize=12)

for tick in ax1.yaxis.get_major_ticks():
    tick.set_pad(10)
for tick in ax1.zaxis.get_major_ticks(): 
    tick.set_pad(10)

plt.title('Power difference [W]', fontsize=16)

filename = measurements_root + os.path.sep + "experiments_power_difference_mean_3d_std.png"
plt.savefig(filename)
filename = measurements_root + os.path.sep + "experiments_power_difference_mean_3d_std.pdf"
plt.savefig(filename)

plt.show()